In [3]:
from src.passenger import *

rid, uid1, ts1, uid2, ts2 = AFC[np.random.choice(len(AFC), 1)].flatten().tolist()
print(rid, uid1, uid2, ts1, ts2)

k_pv = K_PV_DICT[(uid1, uid2)]
k_pv

355809 1086 1111 30171 31672


array([[1086111101,      10336,      10329,          3,         -1],
       [1086111101,      10433,      10434,          4,          1],
       [1086111102,      10336,      10333,          3,         -1],
       [1086111102,      10129,      10125,          1,         -1],
       [1086111102,      10431,      10434,          4,          1],
       [1086111103,      10336,      10330,          3,         -1],
       [1086111103,      10236,      10235,          2,         -1],
       [1086111103,      10126,      10125,          1,         -1],
       [1086111103,      10431,      10434,          4,          1]],
      dtype=int64)

In [5]:
iti_list = find_feas_iti(k_pv, ts1, ts2)

data= []
for iti_id, itinerary in enumerate(iti_list, start=1):
    path_id = itinerary[0]
    for seg_id, (train_id, board_ts, alight_ts) in enumerate(itinerary[1:], start=1):
        data.append([rid, iti_id, path_id, seg_id, train_id, board_ts, alight_ts])
data

[[355809, 1, 1086111101, 1, 10301353, 30421, 31289],
 [355809, 1, 1086111101, 2, 10401784, 31460, 31568],
 [355809, 2, 1086111101, 1, 10301531, 30189, 31057],
 [355809, 2, 1086111101, 2, 10401784, 31460, 31568],
 [355809, 3, 1086111101, 1, 10301531, 30189, 31057],
 [355809, 3, 1086111101, 2, 10401859, 31270, 31378],
 [355809, 4, 1086111101, 1, 10301531, 30189, 31057],
 [355809, 4, 1086111101, 2, 10402045, 31075, 31183],
 [355809, 5, 1086111102, 1, 10301531, 30189, 30555],
 [355809, 5, 1086111102, 2, 10100236, 30600, 31021],
 [355809, 5, 1086111102, 3, 10401784, 31208, 31568],
 [355809, 6, 1086111102, 1, 10301531, 30189, 30555],
 [355809, 6, 1086111102, 2, 10100572, 30730, 31151],
 [355809, 6, 1086111102, 3, 10401784, 31208, 31568]]

In [6]:
read_data("AFC_feas_iti_not_found")

[INFO] read_data(('AFC_feas_iti_not_found',), {}) executing at 04-04 17:24:42.
[INFO] read_data(('AFC_feas_iti_not_found',), {}) executed in  04-04 17:24:42. (04-04 17:24:42 -> 0.0069s )


,rid,uid1,ts1,uid2,ts2
0,132,1127,15042,1101,23955
1,284,1027,18475,1038,21878
2,285,1120,18503,1010,27824
3,286,1120,18506,1010,27824
4,287,1120,18525,1024,25613
...,...,...,...,...,...
92,454,1101,20938,1126,22144
93,455,1101,20940,1126,22140
94,456,1098,20941,1042,21690
95,457,1101,20948,1048,22919
